# Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ssl
from PIL import Image
import json
import requests
import time
from IPython.display import display, clear_output
from datetime import datetime

# Decode 2 bytes bytes pf RGB565 formatted number to pixel, place in matrix to form image and resize. 

In [ ]:




# Fetch the frame size from the URL
def get_frame_size(url):
    response = requests.get(url)
    if response.status_code == 200:
        content = response.text
        width = int(content.split('Width: ')[1].split(',')[0])
        height = int(content.split('Height: ')[1].split(',')[0])
        return width, height
    else:
        raise Exception("Failed to fetch frame size")

# Fetch the file from the URL
ip = "10.162.16.189"
stream_url = 'http://'+ ip +'/stream'
size_url = 'http://'+ ip +'/size'

def process_image(raw_data, width, height):
    display(f"🔄 Processing {width}x{height} image...")
    
    try:
        # Save raw data
        with open("received_image.raw", "wb") as f:
            f.write(raw_data)
        display("💾 Raw image saved")

        # Create image array with dynamic dimensions
        image = np.zeros((height, width, 3), dtype=np.uint8)

        for i in range(width * height):
            # Extract RGB565 data
            msb = raw_data[2 * i]
            lsb = raw_data[2 * i + 1]
            rgb565 = (msb << 8) | lsb

            # Extract RGB components (5:6:5 format)
            red   = (rgb565 >> 11) & 0b11111
            green = (rgb565 >> 5)  & 0b111111
            blue  = rgb565 & 0b11111
            
            # Scale to 8-bit color
            r = red   * 255 // 31
            g = green * 255 // 63
            b = blue  * 255 // 31
            
            # Calculate pixel position
            y = i // width
            x = i % width
            image[y, x] = [r, g, b]

        
        plt.imshow(image)
#        plt.axis('off')
#        plt.title(f'Received Image ({width}x{height})')
        plt.show()

        # Save processed image
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        Image.fromarray(image).save(f"image_{timestamp}.png")
        display(f"Image processed and saved as PNG with timestamp: image_{timestamp}.png")

    except Exception as e:
        display(f"Error processing image: {str(e)}")

# Receive image transmission over HTTP and Display it

In [ ]:

while 1:
    time.sleep(5)
    clear_output(wait=True)
    try:
        # Set a timeout for the request
        width, height = get_frame_size(size_url)
        response = requests.get(stream_url, stream=True, timeout=5)  # Timeout set to 5 seconds

        # **Callback Function: Executes When a Message is Received**
        if response.status_code == 200:
            file_data = bytearray()
            try:
                for chunk in response.iter_content(chunk_size=1024):
                    if chunk:
                        file_data.extend(chunk)
                process_image(file_data, width, height)
            except requests.exceptions.ChunkedEncodingError as e:
                display(f"Chunked encoding error: {str(e)}")
            except Exception as e:
                display(f"Error receiving data: {str(e)}")
        else:
            display(f"Failed to fetch stream: HTTP {response.status_code}")

    except requests.exceptions.Timeout:
        display("Request timed out after 5 seconds")
    except Exception as e:
        display(f"An error occurred: {str(e)}")  

KeyboardInterrupt: 